In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns
import time
import xgboost as xgb
from unidecode import unidecode
from category_encoders import TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [50]:
import category_encoders as ce
import numpy as np
import pandas as pd
from unidecode import unidecode


class data_processing():

    def __init__(self):
        pass

    def procesado_lesionados(self, df):
        '''Coge el dataframe de lesionados y le aplica un OneHotEncoder, pero sin usar la librería. Para tener en cuenta que jugadores
        han participado en el encuentro de inicio o no. Es representativo ya que la no presencia de un jugador puede afectar en el resultado
        de un partido'''
        #Añade una fila de 1 para identificar que ese jugador ha estado lesionado en algún momento
            
        df['lesionados'] = pd.Series(np.ones(len(df)), index=df.index)
        
        
        df_lesionados_id = df[['fixture_id', 'id_lesionado', 'lesionados']]
        
        #Elimino fila si hay missings en la columna de id_jugador_titular
        df_lesionados_id = df_lesionados_id.dropna(subset=['id_lesionado'])
        
        #Pivota la tabla para convertir en una variable cada jugador. Para los partidos que el jugador no ha estado lesionado,
        #se rellenará con un '0'. Dejo el índice de la forma correcta.
        df_lesionados_id = df_lesionados_id.pivot( index= 'fixture_id', 
                                                columns = 'id_lesionado', 
                                                values = 'lesionados').fillna(0).reset_index()
        df_lesionados_id.columns.name = None
        
        #Transformo los valores '1' y '0' a int.
        df_lesionados_id.iloc[:,1:] =df_lesionados_id.iloc[:,1:].astype(int)
        
        #Añado al nombre de las variables de los id de jugadores 'les-' para identificar que es la variable de lesionados.
        df_lesionados_id = df_lesionados_id.rename(columns={col: f'les-{col}' for col in df_lesionados_id.iloc[:,1:]})
        
        return df_lesionados_id


    def procesado_titulares(self, df):
        '''Coge el dataframe de alineaciones y le aplica un OneHotEncoder, pero sin usar la librería. Para tener en cuenta que jugadores
        han participado en el encuentro de inicio o no. Es representativo ya que la no presencia de un jugador puede afectar en el resultado
        de un partido'''    

        #Añade una fila de 1 para identificar que ese jugador ha estado lesionado en algún momento
        df['titular'] = pd.Series(np.ones(len(df)), index=df.index)
        df_alineaciones_id = df[['fixture_id', 'id_jugador_titular', 'titular']]
        #Elimino fila si hay missings en la columna de id_jugador_titular
        df_alineaciones_id = df_alineaciones_id.dropna(subset=['id_jugador_titular'])
        #Pivota la tabla para convertir en una variable cada jugador. Para los partidos que el jugador no ha estado lesionado,
        #se rellenará con un '0'. Dejo el índice de la forma correcta.
        df_alineaciones_id = df_alineaciones_id.pivot( index= 'fixture_id', 
                                                columns = 'id_jugador_titular', 
                                                values = 'titular').fillna(0).reset_index()
        df_alineaciones_id.columns.name = None
        
        #Me cargo un jugador con id nulo (hay que revisarlo después del procesado)
        df_alineaciones_id = df_alineaciones_id.drop(df_alineaciones_id.columns[1], axis=1)
        
        #Transformo los valores '1' y '0' a int.
        df_alineaciones_id.iloc[:,1:]=df_alineaciones_id.iloc[:,1:].astype(int)
        
        #Añado al nombre de las variables de los id de jugadores 'les-' para identificar que es la variable de lesionados.
        df_alineaciones_id = df_alineaciones_id.rename(columns={col: f'titu-{col}' for col in df_alineaciones_id.iloc[:,1:]})
        
        return df_alineaciones_id
    
    def procesado_estadisticas(self, df):
        '''Procesado de todas las estadisticas que se han extraido, y que ocurren dentro de un partido.'''
        #Elimino las filas en las que la API no me devuelve un solo valor(ha pasado)
        rows_with_all_missing = df.iloc[:, :-1].isna().all(axis=1)
        df = df[~rows_with_all_missing]
        #Renombro dos columnas mal nombradas (no es el dato que dice la columna)
        df = df.rename(columns={'pass_precision_local': 'total_pass_local',
                                'pass_precision_away': 'total_pass_away',
                            'fixture_id_2': 'fixture_id'})
        #Transformo los datos de posesion a float para poder usarlos de forma más sencilla
        df['ball_possession_local'] = df['ball_possession_local'].str.replace('%','').astype(float)
        df['ball_possession_away'] = df['ball_possession_away'].str.replace('%', '').astype(float)

        df['ball_possession_local'] = df['ball_possession_local']/100
        df['ball_possession_away'] = df['ball_possession_away']/100
        # Convertir columna a tipo numérico. Esto es porque en las tarjetas amarillas habia datos erroneos (con porcntaje)
        df['yellow_cards_local'] = pd.to_numeric(df['yellow_cards_local'], errors='coerce')
        df['yellow_cards_away'] = pd.to_numeric(df['yellow_cards_away'], errors='coerce')

        # Filtro filas con NaN en la columna en cuestión y las elimino
        rows_with_nan = df['yellow_cards_local'].isna() | df['yellow_cards_away'].isna()
        df = df[~rows_with_nan]
        #Cambio los missings por 0, ya que cuando el valor es 0 la api devuelve null.
        df.fillna(0, inplace = True)
        #Cambio todas las columnas que quiero que sean número entero para trabajar mejor con ellos
        cols_to_int = ['shots_on_goal_local', 'shots_on_goal_away', 'shots_off_goal_local', 'shots_off_goal_away', 
                'total_shots_local', 'total_shots_away', 'blocked_shots_local', 'blocked_shots_away', 
                'shots_insidebox_local', 'shots_insidebox_away', 'shots_outsidebox_local', 'shots_outsidebox_away', 
                'fouls_local', 'fouls_away', 'corners_local', 'corners_away', 'offsides_local', 'offsides_away', 
                'yellow_cards_local', 'yellow_cards_away', 'red_cards_local', 'red_cards_away', 'goalkeeper_saves_local', 
                'goalkeeper_saves_away', 'total_pass_local', 'total_pass_away']
        df[cols_to_int] = df[cols_to_int].astype(int)
        
        return df
        
    def procesado_datos_generales(self, df):
        '''Eliminación de missings en los goles (ya que si no hay goles lo considera como missing), y cambiar el tipo de los goles
        a favor/en contra'''
        #Sustituimos los missings por 0, ya que esos missings significa que ha habido 0 goles
        df['goles_descanso_local'] = df['goles_descanso_local'].fillna(0)
        df['goles_descanso_visitante'] = df['goles_descanso_visitante'].fillna(0)
        #Cambio el tipo de float a int, ya que no puede haber goles decimales
        df['goles_descanso_local'] = df['goles_descanso_local'].astype(int)
        df['goles_descanso_visitante'] = df['goles_descanso_visitante'].astype(int)
        
        return df
    
    def ruta_cuotas(self):
        return [
        '../data/raw_files/cuotas/SP1-2012.csv',
        '../data/raw_files/cuotas/SP2-2012.csv',
        '../data/raw_files/cuotas/SP1-2013.csv',
        '../data/raw_files/cuotas/SP2-2013.csv',
        '../data/raw_files/cuotas/SP1-2014.csv',
        '../data/raw_files/cuotas/SP2-2014.csv',
        '../data/raw_files/cuotas/SP1-2015.csv',
        '../data/raw_files/cuotas/SP2-2015.csv',
        '../data/raw_files/cuotas/SP1-2016.csv',
        '../data/raw_files/cuotas/SP2-2016.csv',
        '../data/raw_files/cuotas/SP1-2017.csv',
        '../data/raw_files/cuotas/SP2-2017.csv',
        '../data/raw_files/cuotas/SP1-2018.csv',
        '../data/raw_files/cuotas/SP2-2018.csv',
        '../data/raw_files/cuotas/SP1-2019.csv',
        '../data/raw_files/cuotas/SP2-2019.csv',
        '../data/raw_files/cuotas/SP1-2020.csv',
        '../data/raw_files/cuotas/SP2-2020.csv',
        '../data/raw_files/cuotas/SP1-2021.csv',
        '../data/raw_files/cuotas/SP2-2021.csv',
        '../data/raw_files/cuotas/SP1-2022.csv',
        '../data/raw_files/cuotas/SP2-2022.csv'
    ]
    
    def procesado_cuotas(self, file_names, df_ids):
        equivalencia_nombres = {
            'Celta':'Celta Vigo',
            'Mallorca':'Mallorca',
            'Sevilla': 'Sevilla',
            'Ath Bilbao': 'Athletic Club',
            'Barcelona':'Barcelona',
            'Levante':'Levante',
            'Real Madrid': 'Real Madrid',
            'La Coruna': 'Deportivo La Coruna',
            'Vallecano': 'Rayo Vallecano',
            'Zaragoza': 'Zaragoza',
            'Betis': 'Real Betis',
            'Espanol':'Espanyol',
            'Malaga':'Malaga',
            'Sociedad':'Real Sociedad',
            'Getafe':'Getafe',
            'Granada':'Granada CF',
            'Osasuna':'Osasuna',
            'Valencia':'Valencia',
            'Ath Madrid':'Atletico Madrid',
            'Valladolid':'Valladolid',
            'Barcelona B':'Barcelona B',
            'Mirandes':'Mirandes',
            'Villarreal':'Villarreal',
            'Girona':'Girona',
            'Lugo':'Lugo',
            'Xerez':'Xerez',
            'Alcorcon':'Alcorcon',
            'Elche':'Elche',
            'Numancia':'Numancia',
            'Santander':'Racing Santander',
            'Murcia':'Real Murcia',
            'Almeria':'Almeria',
            'Guadalajara':'Guadalajara',
            'Huesca':'Huesca',
            'Las Palmas':'Las Palmas',
            'Ponferradina':'Ponferradina',
            'Real Madrid B':'Real Madrid II',
            'Recreativo':'Recreativo Huelva',
            'Sabadell':'Sabadell',
            'Sp Gijon':'Sporting Gijon',
            'Cordoba':'Cordoba',
            'Hercules':'Hércules',
            'Jaen':'Real Jaén',
            'Alaves':'Alaves',
            'Eibar':'Eibar',
            'Tenerife':'Tenerife',
            'Albacete':'Albacete',
            'Leganes':'Leganes',
            'Llagostera':'Llagostera',
            'Gimnastic':'Gimnastic',
            'Oviedo':'Oviedo',
            'Ath Bilbao B':'Athletic Club II',
            'Sevilla B':'Sevilla Atletico',
            'Reus Deportiu':'Reus',
            'Cadiz':'Cadiz',
            'UCAM Murcia':'Ucam Murcia',
            'Lorca':'Lorca',
            'Leonesa':'Cultural Leonesa',
            'Extremadura UD':'Extremadura',
            'Rayo Majadahonda':'Rayo Majadahonda',
            'Fuenlabrada':'Fuenlabrada',
            'Castellon':'Castellón',
            'Cartagena':'FC Cartagena',
            'Logrones':'UD Logroñés',
            'Sociedad B':'Real Sociedad II',
            'Ibiza':'Ibiza',
            'Amorebieta':'Amorebieta',
            'Burgos':'Burgos',
            'Villarreal B':'Villarreal II',
            'Andorra':'FC Andorra'
        }
    
        def select_columns_and_add_season(df, file_name):
            # Extraer año del nombre del archivo
            year = file_name.split('-')[1][:4]

            # Crear un diccionario que contenga los nombres de los equipos como claves y sus IDs como valores
            equipo_id = {}
            for index, row in df_ids.iterrows():
                equipo_id[row['equipo_jugador']] = row['id_equipo']

            # Reemplazar los nombres de los equipos por sus IDs correspondientes utilizando el diccionario de equivalencias y el diccionario equipo_id
            df['HomeTeam'] = df['HomeTeam'].map(equivalencia_nombres).map(equipo_id)
            df['AwayTeam'] = df['AwayTeam'].map(equivalencia_nombres).map(equipo_id)

            # Seleccionar columnas requeridas
            df_selected = df[['HomeTeam', 'AwayTeam', 'B365H', 'B365D', 'B365A']]

            # Añadir columna "season" con el año extraído
            df_selected['season'] = int(year)

            # Eliminar filas con valores NaN
            df_selected.dropna(inplace=True)

            return df_selected

        # Lista para guardar los dataframes procesados
        processed_dfs = []

        # Iterar sobre los nombres de archivo
        for file_name in file_names:
            # Leer archivo CSV en un dataframe
            df = pd.read_csv(file_name)

            # Aplicar la función select_columns_and_add_season y renombrar las columnas
            df_processed = select_columns_and_add_season(df, file_name).rename(columns={'B365H': 'odd_1', 'B365D': 'odd_x', 'B365A': 'odd_2'})

            # Agregar el dataframe procesado a la lista
            processed_dfs.append(df_processed)

        # Concatenar todos los dataframes procesados en uno solo
        final_df = pd.concat(processed_dfs, ignore_index=True)
        final_df = final_df.dropna(how='any')
        final_df['HomeTeam'] = final_df['HomeTeam'].astype(int)
        final_df['AwayTeam'] = final_df['AwayTeam'].astype(int)

        return final_df
        
    def creacion_df_final(self, df_lesionados, df_alineaciones, df_datos_partidos, df_estadisticas, df_cuotas):
        '''Esta función hace un merge de todos los datos sacados anteriormente'''
        #Comenzamos la unión de dataframes, empezando por los datos de partidos y estadísticas
        df_final = pd.merge(df_datos_partidos, df_estadisticas, on='fixture_id', how='left')
        
        #Elimino las filas en las que la API no me devuelve un solo valor(ha pasado)
        rows_with_all_missing = df_final.loc[:, 'shots_on_goal_local':].isna().all(axis=1)
        df_final = df_final[~rows_with_all_missing]
        
        #Unimos el df resultante con el de lesionados
        df_final = pd.merge(df_final,df_lesionados, on='fixture_id', how = 'left')
        #Relleno los missings con 0, ya que significa que en esos partidos no ha habido lesionados
        df_final = df_final.fillna(0)
        
        #Unimos el df_final con el de alineaciones, que es el que faltaría.
        df_final = pd.merge(df_final, df_alineaciones, on='fixture_id', how='left')
        #Relleno los missings con 0, ya que significa que en esos partidos no habría participado ese jugador
        df_final = df_final.fillna(0)
        
        #Unimos el df_final con el de odds
        df_final = pd.merge(df_final, df_cuotas, left_on=['id_equipo_local', 'id_equipo_visitante', 'season'], 
                            right_on=['HomeTeam', 'AwayTeam', 'season'], how='inner')
        
        #Eliminamos filas con NaN
        df_final = df_final.dropna(how='any')
        df_final = df_final.drop(['HomeTeam','AwayTeam'], axis=1)
        
        
        df_final = df_final.reset_index()

        #Para agilizar tiempos en métedos que necesitan esta tabla para usarse, ya que tarda un poco en ejecutarse.
        #df_final.to_csv('df_partidos_completo.csv', index=False)


        
        return df_final
        
    



    #Las funciones siguientes tendrán únicamente la utilidad de crear datos nuevos.
    def buscar_jugador(self, id_equipo, temporada_equipo):
        ''' Esta función únicamente será llamada para localizar los ids de jugadores y poder crear los datos nuevos'''

        df = pd.read_csv("df_diccionario_jugadores.csv")

        # Filtre el DataFrame original utilizando los valores de los parámetros
        filtro = (df['id_equipo'] == id_equipo) & (df['temporada_equipo'] == temporada_equipo)
        df_filtrado = df[filtro]

        # Devuelva el DataFrame filtrado
        return df_filtrado

    def buscar_equipo(self, nombres):
        ''' Esta función únicamente será llamada para localizar los ids delos equipos y poder crear los datos nuevos. Acepta una lista de nombres o
         un único nombre '''
        # Cargo el diccionario de ids que tengo y fue descargado. Es el mismo que de jugadores
        equipos = pd.read_csv("df_diccionario_jugadores.csv")
        # Elimino acentos de los nombres de los equipos en el DataFrame aplicando unidecode. También se quedan en minúsculas
        equipos['nombre_equipo'] = equipos['equipo_jugador'].apply(lambda x: unidecode(x.lower()))
        
        if isinstance(nombres, str):
            # Eliminar las marcas diacríticas del nombre introducido
            equipo = unidecode(nombres.lower())
            # Busco los equipos cuyo nombre contenga la cadena de texto introducida como parámetro
            equipos_coincidentes = equipos[equipos['nombre_equipo'].str.contains(nombres, case=False)]
            #Elimino los jugadores duplicados, porque pueden salir jugadores repetidos si participaron más de 1 temporada
            equipos_coincidentes = equipos_coincidentes.drop_duplicates(subset='id_equipo')
            # Devuelve una tabla con los nombres y los ids de los jugadores encontrados
            return equipos_coincidentes[['nombre_equipo', 'id_equipo']]
        
        elif isinstance(nombres, list):
            resultados = []
            # Busco cada id de los jugadores en la lista
            for n in nombres:
                # Elimino acentos del nombre introducido
                equipo = unidecode(n.lower())
                # Busco los jugadores cuyo nombre contenga la cadena de texto introducida como parámetro
                equipos_coincidentes = equipos[equipos['nombre_equipo'].str.contains(n, case=False)]
                # Elimino jugadores duplicados en función del id
                equipos_coincidentes = equipos_coincidentes.drop_duplicates(subset='id_equipo')
                # Añado los resultados a la lista de resultados
                for i, row in equipos_coincidentes.iterrows():
                    resultados.append([row['nombre_equipo'], row['id_equipo']])
            # Devuelvo una tabla con los nombres y los IDs de los jugadores encontrados
            return pd.DataFrame(resultados, columns=['nombre_equipo', 'id_equipo'])
        
        else: return 'Introduce una lista de nombres o un nombre único'

    
    def nombre_arbitro_correcto(self, nombre):

        arbitros = pd.read_csv('df_partidos_completo.csv')
        # Elimino los acentos del nombre introducido
        arbitro = unidecode(nombre.lower())
        # Busco los equipos cuyo nombre contenga la cadena de texto introducida como parámetro
        arbitros_coincidentes = arbitros[arbitros['arbitro'].str.contains(nombre, case=False)]
        #Elimino los arbitros duplicados, porque pueden salir árbitros repetidos si participaron más de 1 temporada
        arbitros_coincidentes = arbitros_coincidentes.drop_duplicates(subset='arbitro')
        if len(arbitros_coincidentes) == 0:
            return None
        else:
            # Obtengo el índice de la fila correspondiente al árbitro
            indice = arbitros_coincidentes.index[0]
            # Obtengo el nombre del árbitro con el formato adecuado
            nombre_completo = arbitros.loc[indice, 'arbitro']
            # Devuelve el nombre del árbitro con el formato adecuado
            return nombre_completo


    def nombre_estadio_correcto(self, nombre):
        estadios = pd.read_csv('df_partidos_completo.csv')
        # Elimino los acentos del nombre introducido
        estadio = unidecode(nombre.lower())
        # Busco los equipos cuyo nombre contenga la cadena de texto introducida como parámetro
        estadios_coincidentes = estadios[estadios['estadio'].str.contains(nombre, case=False)]
        #Elimino los arbitros duplicados, porque pueden salir estadios repetidos si participaron más de 1 temporada
        estadios_coincidentes = estadios_coincidentes.drop_duplicates(subset='estadio')
        if len(estadios_coincidentes) == 0:
            return None
        else:
            # Obtengo el índice de la fila correspondiente al estadio
            indice = estadios_coincidentes.index[0]
            # Obtengo el nombre del estadio con el formato adecuado
            nombre_completo = estadios.loc[indice, 'estadio']
            # Devuelve el nombre del estadio con el formato adecuado
            return nombre_completo
    
    def creacion_datos_nuevos(self, df_partidos,id_equipo_local, id_equipo_visitante,odd_1, odd_x, odd_2, arbitro, estadio, season, ids_lesionados, ids_titulares):
        #Leo el csv donde estan todos los datos completos de los partidos. Se ha creado con la función creacion_df_final()
        
        #Creo los datos de estadisticas que se preveen con la media de datos de los últimos 3 partidos en casa o de visitante
        shots_on_goal_local = np.mean(df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'shots_on_goal_local'].shift(1) +  \
                                df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'shots_on_goal_local'].shift(2) +  \
                                df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'shots_on_goal_local'].shift(3))
        shots_on_goal_away = np.mean(df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'shots_on_goal_away'].shift(1) +  \
                                    df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'shots_on_goal_away'].shift(2) +  \
                                    df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'shots_on_goal_away'].shift(3))
        shots_off_goal_local = np.mean(df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'shots_off_goal_local'].shift(1) +  \
                                    df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'shots_off_goal_local'].shift(2) +  \
                                    df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'shots_off_goal_local'].shift(3))
        shots_off_goal_away = np.mean(df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'shots_off_goal_away'].shift(1) +  \
                                    df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'shots_off_goal_away'].shift(2) +  \
                                    df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'shots_off_goal_away'].shift(3))
        total_shots_local = np.mean(df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'total_shots_local'].shift(1) +  \
                                    df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'total_shots_local'].shift(2) +  \
                                    df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'total_shots_local'].shift(3))
        total_shots_away = np.mean(df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'total_shots_away'].shift(1) +  \
                                    df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'total_shots_away'].shift(2) +  \
                                    df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'total_shots_away'].shift(3))
        blocked_shots_local = np.mean(df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'blocked_shots_local'].shift(1) +  \
                                    df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'blocked_shots_local'].shift(2) +  \
                                    df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'blocked_shots_local'].shift(3))
        blocked_shots_away = np.mean(df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'blocked_shots_away'].shift(1) +  \
                                    df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'blocked_shots_away'].shift(2) +  \
                                    df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'blocked_shots_away'].shift(3))
        shots_insidebox_local = np.mean(df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'shots_insidebox_local'].shift(1) +  \
                                    df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'shots_insidebox_local'].shift(2) +  \
                                    df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'shots_insidebox_local'].shift(3))
        shots_insidebox_away = np.mean(df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'shots_insidebox_away'].shift(1) +  \
                                    df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'shots_insidebox_away'].shift(2) +  \
                                    df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'shots_insidebox_away'].shift(3))
        shots_outsidebox_local = np.mean(df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'shots_outsidebox_local'].shift(1) +  \
                                    df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'shots_outsidebox_local'].shift(2) +  \
                                    df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'shots_outsidebox_local'].shift(3))
        shots_outsidebox_away = np.mean(df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'shots_outsidebox_away'].shift(1) +  \
                                    df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'shots_outsidebox_away'].shift(2) +  \
                                    df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'shots_outsidebox_away'].shift(3))
        fouls_local = np.mean(df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'fouls_local'].shift(1) +  \
                                    df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'fouls_local'].shift(2) +  \
                                    df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'fouls_local'].shift(3))
        fouls_away = np.mean(df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'fouls_away'].shift(1) +  \
                                    df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'fouls_away'].shift(2) +  \
                                    df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'fouls_away'].shift(3))
        corners_local = np.mean(df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'corners_local'].shift(1) +  \
                                    df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'corners_local'].shift(2) +  \
                                    df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'corners_local'].shift(3))
        corners_away = np.mean(df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'corners_away'].shift(1) +  \
                                    df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'corners_away'].shift(2) +  \
                                    df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'corners_away'].shift(3))
        offsides_local = np.mean(df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'offsides_local'].shift(1) +  \
                                    df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'offsides_local'].shift(2) +  \
                                    df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'offsides_local'].shift(3))
        offsides_away = np.mean(df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'offsides_away'].shift(1) +  \
                                    df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'offsides_away'].shift(2) +  \
                                    df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'offsides_away'].shift(3))
        ball_possession_local = np.mean(df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'ball_possession_local'].shift(1) +  \
                                    df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'ball_possession_local'].shift(2) +  \
                                    df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'ball_possession_local'].shift(3))
        ball_possession_away = np.mean(df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'ball_possession_away'].shift(1) +  \
                                    df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'ball_possession_away'].shift(2) +  \
                                    df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'ball_possession_away'].shift(3))
        yellow_cards_local = np.mean(df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'yellow_cards_local'].shift(1) +  \
                                    df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'yellow_cards_local'].shift(2) +  \
                                    df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'yellow_cards_local'].shift(3))
        yellow_cards_away = np.mean(df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'yellow_cards_away'].shift(1) +  \
                                    df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'yellow_cards_away'].shift(2) +  \
                                    df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'yellow_cards_away'].shift(3))
        red_cards_local = np.mean(df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'red_cards_local'].shift(1) +  \
                                    df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'red_cards_local'].shift(2) +  \
                                    df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'red_cards_local'].shift(3))
        red_cards_away = np.mean(df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'red_cards_away'].shift(1) +  \
                                    df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'red_cards_away'].shift(2) +  \
                                    df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'red_cards_away'].shift(3))
        goalkeeper_saves_local = np.mean(df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'goalkeeper_saves_local'].shift(1) +  \
                                    df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'goalkeeper_saves_local'].shift(2) +  \
                                    df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'goalkeeper_saves_local'].shift(3))
        goalkeeper_saves_away = np.mean(df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'goalkeeper_saves_away'].shift(1) +  \
                                    df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'goalkeeper_saves_away'].shift(2) +  \
                                    df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'goalkeeper_saves_away'].shift(3))
        total_pass_local = np.mean(df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'total_pass_local'].shift(1) +  \
                                    df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'total_pass_local'].shift(2) +  \
                                    df_partidos.loc[df_partidos['id_equipo_local'] == id_equipo_local, 'total_pass_local'].shift(3))
        total_pass_away = np.mean(df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'total_pass_away'].shift(1) +  \
                                    df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'total_pass_away'].shift(2) +  \
                                    df_partidos.loc[df_partidos['id_equipo_visitante'] == id_equipo_visitante, 'total_pass_away'].shift(3))
        
        #Creo un dataframe de solo estadísticas más los datos de arbitro, estadio, ids, season (todos menos lesionados y titulares)
        df_datos_nuevos = pd.DataFrame({
                                        'id_equipo_local': id_equipo_local,
                                            'id_equipo_visitante': id_equipo_visitante,
                                            'arbitro': arbitro,
                                            'estadio': estadio,
                                            'season': season,
                                        'shots_on_goal_local':shots_on_goal_local,
                                        'shots_on_goal_away':shots_on_goal_away,
                                        'shots_off_goal_local':shots_off_goal_local,
                                        'shots_off_goal_away':shots_off_goal_away,
                                        'total_shots_local':total_shots_local,
                                        'total_shots_away':total_shots_away,
                                        'blocked_shots_local':blocked_shots_local,
                                        'blocked_shots_away':blocked_shots_away,
                                        'shots_insidebox_local':shots_insidebox_local,
                                        'shots_insidebox_away':shots_insidebox_away,
                                        'shots_outsidebox_local':shots_outsidebox_local,
                                        'shots_outsidebox_away':shots_outsidebox_away,
                                        'fouls_local':fouls_local,
                                        'fouls_away':fouls_away,
                                        'corners_local':corners_local,
                                        'corners_away':corners_away,
                                        'offsides_local':offsides_local,
                                        'offsides_away':offsides_away,
                                        'ball_possession_local':ball_possession_local,
                                        'ball_possession_away':ball_possession_away,
                                        'yellow_cards_local':yellow_cards_local,
                                        'yellow_cards_away':yellow_cards_away,
                                        'red_cards_local':red_cards_local,
                                        'red_cards_away':red_cards_away,
                                        'goalkeeper_saves_local':goalkeeper_saves_local,
                                        'goalkeeper_saves_away':goalkeeper_saves_away,
                                        'total_pass_local':total_pass_local,
                                        'total_pass_away':total_pass_away
                                        }, index = [0])
        
        #Creo un dataframe solo con las columnas de lesionados y las relleno con 0 todas
        #Primero localizo todas las columnas de df_partidos
        columns_les = []
        for col in df_partidos.columns:
            if 'les-' in col:
                columns_les.append(col)
        #Relleno con 0 y creo el dataframe de lesionados
        valores = {col: 0 for col in columns_les}
        df_lesionados_nuevos = pd.DataFrame([valores])
        
        #Hago el mismo proceso con un dataframe de titulares
        columns_titus = []
        for col in df_partidos.columns:
            if 'titu-' in col:
                columns_titus.append(col)
        valores_titu = {col: 0 for col in columns_titus}
        df_titulares_nuevos = pd.DataFrame([valores_titu])
        
        #Concateno los 3 dataframe para obtener el dataframe de datos final
        df_datos_nuevos_final = pd.concat([df_datos_nuevos, df_lesionados_nuevos,df_titulares_nuevos], axis = 1)
        
        #Añado los prefijos y sufijos necesarios para localizar los ids de lesionados y titulares en la tabla
        ids_lesionado_prefijo = ['les-{}'.format(id) for id in ids_lesionados]
        ids_titular_prefijo = ['titu-{}{}'.format(id,'.0') for id in ids_titulares]
        
        #Y sustituyo el valor correspondiente por 1, ya que o estan lesionados en ese partido o van a jugar
        for id_les in ids_lesionado_prefijo:
            df_datos_nuevos_final.loc[0, id_les] = 1
        for id_titu in ids_titular_prefijo:
            df_datos_nuevos_final.loc[0, id_titu] = 1
            
        
        df_datos_nuevos_final['odd_1'] = odd_1
        df_datos_nuevos_final['odd_x'] = odd_x
        df_datos_nuevos_final['odd_2'] = odd_2

        return df_datos_nuevos_final

In [51]:
data_processing = data_processing()

In [55]:
df_datos_generales = pd.read_csv('../data/raw_files/datos_generales_fx.csv')
df_estadisticas = pd.read_csv('../data/raw_files/df_estadisticas.csv')
df_alineaciones = pd.read_csv('../data/raw_files/datos_alineaciones.csv')
df_lesionados = pd.read_csv('../data/raw_files/datos_lesionados.csv')
df_dicc_equipos = pd.read_csv('../data/raw_files/df_dicc_equipos.csv')
cuotas = data_processing.ruta_cuotas()

In [56]:
len(df_datos_generales)

7155

In [32]:
df_datos_generales_procesado = data_processing.procesado_datos_generales(df_datos_generales)

#Procesamiento de las estadisticas de los partidos
df_estadisticas_procesado = data_processing.procesado_estadisticas(df_estadisticas)
#Procesamiento de las alineaciones de los partidos
df_alineaciones_procesado = data_processing.procesado_titulares(df_alineaciones)
#Procesamiento de los lesionados de los partidos
df_lesionados_procesado = data_processing.procesado_lesionados(df_lesionados)
#Procesamiento de los datos de las cuotas
df_cuotas_procesado = data_processing.procesado_cuotas(cuotas,df_dicc_equipos)
#Unión de los 4 dataframes anteriores, y realización de limpieza e imputación de missings si los hubiera
df_union_procesado = data_processing.creacion_df_final(df_lesionados=df_lesionados_procesado, 
                                                       df_alineaciones=df_alineaciones_procesado,
                                                        df_datos_partidos=df_datos_generales_procesado,
                                                        df_estadisticas=df_estadisticas_procesado,
                                                        df_cuotas = df_cuotas_procesado)
#Creación de nuevas variables interesantes para el desempeño del modelo
#df_final = data_processing.creacion_nuevas_variables(df_union_procesado)

C:\Users\gonve\AppData\Local\Temp\ipykernel_46484\3417217326.py:234: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['season'] = int(year)
C:\Users\gonve\AppData\Local\Temp\ipykernel_46484\3417217326.py:237: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected.dropna(inplace=True)
C:\Users\gonve\AppData\Local\Temp\ipykernel_46484\3417217326.py:234: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [33]:
df_union_procesado.head()

,index,id_equipo_local,id_equipo_visitante,goles_local,goles_visitante,resultado,arbitro,fixture_id,fecha_timestamp,goles_descanso_local,...,titu-337011.0,titu-337031.0,titu-337523.0,titu-338295.0,titu-341700.0,titu-347886.0,titu-380261.0,odd_1,odd_x,odd_2
0,0,8157,534,0,0,0,"Jose Antonio Lopez Toca, Spain",878641,1677873600,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.75,2.9,2.88
1,1,530,536,6,1,1,"Guillermo Cuadra Fernandez, Spain",878172,1677960000,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.67,3.6,5.25
2,2,546,547,3,2,1,"Juan Martinez Munuera, Spain",878175,1677934800,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.75,3.0,2.80
3,3,723,533,0,2,2,"Pablo Gonzales Fuertes, Spain",878176,1677942900,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.50,3.6,2.05
4,4,798,797,0,1,2,"José Luis Munuera Montero, Spain",878179,1677951000,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.70,3.4,5.75


In [34]:
df_final = pd.read_csv('../data/processed_files/df_datos_completos.csv')

In [35]:
#Dividimos en los datos de entrenamiento y la clasificación de los datos de entrenamiento que usaremos para entrenar el modelo
X = df_final.drop(['index','fixture_id','resultado', 'goles_local', 'goles_visitante','goles_descanso_local','goles_descanso_visitante','fecha_timestamp' ], axis=1)
y = df_final['resultado']



In [40]:
# Pipeline para codificar la columna 'arbitro' con OneHotEncoder
arbitro_pipeline = Pipeline([
    ('onehot', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

# Pipeline para codificar la columna 'estadio' con TargetEncoder
estadio_pipeline = Pipeline([
    ('target', TargetEncoder())
])

# ColumnTransformer para aplicar los pipelines a las columnas correspondientes
preprocessor = ColumnTransformer([
    ('arbitro', arbitro_pipeline, ['arbitro']),
    ('estadio', estadio_pipeline, ['estadio']),
    ], remainder = "passthrough")

# Pipeline final con el preprocesamiento y el modelo RandomForestClassifier
pipeline_xgb = Pipeline([
    ('preprocessor', preprocessor),
    ('pca', PCA()),
    ('xgb', xgb.XGBClassifier())
])

xgb_param = {
'pca__n_components': [12,30,35],
'xgb__n_estimators': [300, 500, 700],
'xgb__learning_rate': [0.1],
'xgb__max_depth': [27,25],
'xgb__subsample': [0.5, 0.8],
'xgb__colsample_bytree': [0.5, 0.6],
'xgb__min_child_weight': [1, 2],
'xgb__gamma': [0]
}

gs_xgb = GridSearchCV(
                        pipeline_xgb,
                        xgb_param,
                        cv=3,
                        scoring="accuracy",
                        verbose=1,
                        n_jobs=-1
                    )

In [43]:
X.info

<bound method DataFrame.info of     id_equipo_local  id_equipo_visitante  \
0              8157                  534   
1               530                  536   
2               546                  547   
3               723                  533   
4               798                  797   
5              9580                  715   
6               726                  539   
7               799                  718   
8               719                  545   
9               720                  540   
10              543                  541   
11              716                  732   
12              535                 4665   
13              722                  731   
14             4907                 5262   
15              537                 9390   
16             9595                  542   

                                   arbitro  \
0           Jose Antonio Lopez Toca, Spain   
1        Guillermo Cuadra Fernandez, Spain   
2             Juan Martinez Munuera, 

In [41]:
gs_xgb.fit(X, y)

Fitting 3 folds for each of 144 candidates, totalling 432 fits


c:\Users\gonve\anaconda3\envs\general\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
384 fits failed out of a total of 432.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
96 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\gonve\anaconda3\envs\general\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\gonve\anaconda3\envs\general\lib\site-packages\sklearn\pipeline.py", line 378, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "c:\Users\gonve\anaconda3\envs\general\lib\site-packages\sklearn\pipeline.py", line 336, in _fit
    X, fitted_transformer = fi

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('arbitro',
                                                                         Pipeline(steps=[('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse=False))]),
                                                                         ['arbitro']),
                                                                        ('estadio',
                                                                         Pipeline(steps=[('target',
                                                                                          TargetEncoder())]),
                                                                         ['estadio'])])),
                                       ('pca', PCA()),
                                       ('xgb',
                                        XGBClassifier(base_score=None,
                                                      booste...
                                                      n_jobs=None,
                                                      num_parallel_tree=None,
                                                      predictor=None,
                                                      random_state=None, ...))]),
             n_jobs=-1,
             param_grid={'pca__n_components': [12, 30, 35],
                         'xgb__colsample_bytree': [0.5, 0.6], 'xgb__gamma': [0],
                         'xgb__learning_rate': [0.1],
                         'xgb__max_depth': [27, 25],
                         'xgb__min_child_weight': [1, 2],
                         'xgb__n_estimators': [300, 500, 700],
                         'xgb__subsample': [0.5, 0.8]},
             scoring='accuracy', verbose=1)